In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("U-data.csv")
df

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Load the dataset
df = pd.read_csv("U-data.csv")

# Encode categorical features
encoder = LabelEncoder()
categorical_columns = ['Dietary_Fiber_Intake', 'Physical_Activity_Level']
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])

# Define features and target
X = df.drop(columns=['Ulcerative_Colitis'])
y = df['Ulcerative_Colitis']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.94
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88        52
           1       0.97      0.95      0.96       148

    accuracy                           0.94       200
   macro avg       0.91      0.92      0.92       200
weighted avg       0.94      0.94      0.94       200



C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:39:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [3]:
import joblib

# Save the model
joblib.dump(xgb_model, 'xgb_ulcerative_colitis_model.pkl')

# Load the model
loaded_model = joblib.load('xgb_ulcerative_colitis_model.pkl')

In [5]:
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the saved model
model = joblib.load('xgb_ulcerative_colitis_model.pkl')

# Initialize LabelEncoder
encoder = LabelEncoder()
encoder.fit(["Low", "Moderate", "High"])  # Match with training data categories

# Classification key for the prediction
classification_key = {
    0: "No Ulcerative Colitis",
    1: "Ulcerative Colitis Detected"
}

def preprocess_input(input_data, encoder):
    """
    Preprocess input data to match the training data structure.
    Args:
        input_data (dict): Input data with keys as feature names.
        encoder (LabelEncoder): Encoder for categorical variables.

    Returns:
        pd.DataFrame: Preprocessed input data.
    """
    # Convert input dictionary to a DataFrame
    input_df = pd.DataFrame([input_data])

    # Encode categorical columns
    for col in ['Dietary_Fiber_Intake', 'Physical_Activity_Level']:
        input_df[col] = encoder.transform(input_df[col])

    return input_df

def predict_ulcerative_colitis(model, input_data, encoder, classification_key):
    """
    Predict Ulcerative Colitis given the input features.

    Args:
        model: Trained XGBoost model.
        input_data (dict): Input data with feature names as keys and their values.
        encoder (LabelEncoder): Encoder for categorical variables.
        classification_key (dict): Mapping of prediction output to human-readable classes.

    Returns:
        str: Human-readable prediction result.
    """
    # Preprocess the input
    processed_input = preprocess_input(input_data, encoder)
    
    # Make prediction
    prediction = model.predict(processed_input)

    # Return human-readable classification
    return classification_key[int(prediction[0])]

# Example Input
input_data = {
    "Age": 69,
    "Family_History": 0,
    "BMI": 25.69,
    "Smoking_History": 0,
    "Dietary_Fiber_Intake": "Moderate",
    "Physical_Activity_Level": "High",
    "Stress_Level": 7,
    "Medication_Use": 1
}

# Make Prediction
prediction = predict_ulcerative_colitis(model, input_data, encoder, classification_key)
print(f"Prediction: {prediction}")

Prediction: No Ulcerative Colitis
